In [1]:
import sys, os
sys.path.insert(0, '/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from trainlib.ConfigFileHandler import ConfigFileHandler
from trainlib.ModelCollectionConfigFileHandler import ModelCollectionConfigFileHandler

Welcome to JupyROOT 6.10/09


Using TensorFlow backend.


In [8]:
campaign_workspace = "/data_CMS/cms/wind/"

In [9]:
def get_loss(run, mcoll, model):
    confhandler = ConfigFileHandler()
    confhandler.load_configuration(os.path.join(campaign_workspace, run, "training", mcoll, "model_benchmark.txt"))
    return float(confhandler.get_field(model, 'val_loss'))

In [51]:
def index_translation_dict(df, column):
    slist = sorted(list(set(df[column])))
    print slist
    return {val: index for index, val in enumerate(slist)}

In [52]:
def convert_to_matshow(df, keys, plotcol):
    translations = []
    for key in keys:
        translations.append(index_translation_dict(df, key))
                
    plot_data = np.zeros((len(translations[0]), len(translations[1]))) 
        
    for index, row in df.iterrows():
        i = translations[0][row[keys[0]]]
        j = translations[1][row[keys[1]]]

        plot_data[i,j] = row[plotcol]
       
    return plot_data, np.array(sorted(translations[0].keys())), np.array(sorted(translations[1].keys()))

In [10]:
input_runs = ["180515_hyperparameter_sweep/number_neurons_16.0number_layers_0.0",
              "180515_hyperparameter_sweep/number_neurons_16.0number_layers_1.0",
              "180515_hyperparameter_sweep/number_neurons_16.0number_layers_2.0",
              "180515_hyperparameter_sweep/number_neurons_16.0number_layers_3.0",
              "180515_hyperparameter_sweep/number_neurons_32.0number_layers_0.0",
              "180515_hyperparameter_sweep/number_neurons_32.0number_layers_1.0",
              "180515_hyperparameter_sweep/number_neurons_32.0number_layers_2.0",
              "180515_hyperparameter_sweep/number_neurons_32.0number_layers_3.0",
              "180516_hyperparameter_sweep/number_neurons_48.0number_layers_0.0",
              "180516_hyperparameter_sweep/number_neurons_48.0number_layers_1.0",
              "180516_hyperparameter_sweep/number_neurons_48.0number_layers_2.0",
              "180516_hyperparameter_sweep/number_neurons_48.0number_layers_3.0",
              "180516_hyperparameter_sweep/number_neurons_64.0number_layers_0.0",
              "180516_hyperparameter_sweep/number_neurons_64.0number_layers_1.0",
              "180516_hyperparameter_sweep/number_neurons_64.0number_layers_2.0",
              "180516_hyperparameter_sweep/number_neurons_64.0number_layers_3.0"]

In [97]:
mcoll_name = "D_ZHl_ggH_ML"
model_name = "D_ZHl_ggH_ML"
pretty_name = r'$s_{\mathrm{ZH-lept, ggH}}$'

In [98]:
df = pd.DataFrame()

for run in input_runs:
    mconfhandler = ModelCollectionConfigFileHandler()
    mconfhandler.load_configuration(os.path.join(campaign_workspace, run, "settings_training", mcoll_name, "settings.conf"))
    mcolls = mconfhandler.GetModelCollection()
    
    row_dict = {}
    for mcoll in mcolls:
        if mcoll.name == mcoll_name:
            row_dict = dict(mcoll.model_dict[model_name].hyperparameters)
            break
    
    loss = get_loss(run, mcoll_name, model_name)
    row_dict["loss"] = loss
    
    df = df.append(row_dict, ignore_index = True)

attempting to load configuration file from /data_CMS/cms/wind/180515_hyperparameter_sweep/number_neurons_16.0number_layers_0.0/settings_training/D_ZHl_ggH_ML/settings.conf
got the following list of hyperparams: {u'number_layers': 0.0, u'number_neurons': 16.0}
building network with 24 inputs
FlexiblePCAPreprocessor set up for 24 inputs
attempting to load configuration file from /data_CMS/cms/wind/180515_hyperparameter_sweep/number_neurons_16.0number_layers_0.0/training/D_ZHl_ggH_ML/model_benchmark.txt
attempting to load configuration file from /data_CMS/cms/wind/180515_hyperparameter_sweep/number_neurons_16.0number_layers_1.0/settings_training/D_ZHl_ggH_ML/settings.conf
got the following list of hyperparams: {u'number_layers': 1.0, u'number_neurons': 16.0}
building network with 24 inputs
FlexiblePCAPreprocessor set up for 24 inputs
attempting to load configuration file from /data_CMS/cms/wind/180515_hyperparameter_sweep/number_neurons_16.0number_layers_1.0/training/D_ZHl_ggH_ML/model_be

In [99]:
data, x_label, y_label = convert_to_matshow(df, ["number_layers", "number_neurons"], "loss")

[0.0, 1.0, 2.0, 3.0]
[16.0, 32.0, 48.0, 64.0]


In [100]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
cax = ax.matshow(data.transpose(), interpolation = 'nearest', cmap = 'RdYlGn_r', vmin = np.min(data), vmax = np.max(data))
ax.set_xticklabels(np.concatenate([[''], x_label.astype(int) + 1]))
ax.set_yticklabels(np.concatenate([[''], y_label.astype(int)]))
ax.xaxis.set_label_position("bottom")
ax.xaxis.tick_bottom()
ax.set_xlabel("hidden layers")
ax.set_ylabel("neurons / hidden layer")
ax.set_title(pretty_name, fontsize = 15)

for cur in np.ndenumerate(data):
    coords = cur[0]
    val = cur[1]
    
    if val > 0.053:
        color = 'black'
    else:
        color = 'white'
    
    ax.text(coords[0], coords[1], "{:4.4f}".format(val), va = 'center', ha = 'center', color = color, weight = 'bold')
    
plt.tight_layout()

In [101]:
plt.savefig("/data_CMS/cms/wind/HyperparameterPlots/ZHl_ggH.pdf")
#plt.show()